<a href="https://www.kaggle.com/code/alibidaran/pothole-detection-yolov5?scriptVersionId=136273276" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
!cp /kaggle/input/pothole-detection-dataset/data.yaml data.yaml 
!cp -R /kaggle/input/pothole-detection-dataset pothole-detection-dataset

In [ ]:
import glob 
import cv2 
from PIL import Image
import yaml 

In [ ]:
config_file_template = '''
train: ../pothole-detection-dataset/train/images
val: ../pothole-detection-dataset/valid/images

nc: 1
names: ['pothole']
'''

with open('/kaggle/working/yolov5/pothole-detection-dataset/data.yaml', 'w') as f:
    f.write(config_file_template)

In [ ]:
with open('/kaggle/working/yolov5/pothole-detection-dataset/data.yaml', 'r') as f:
    file_path=yaml.safe_load(f)
print(file_path)

In [ ]:
image_list=glob.glob("/kaggle/input/pothole-detection-dataset/train/images/*")
img=cv2.imread(image_list[0])
img_shape=img.shape
img_shape
dataset_location="/kaggle/working/yolov5/pothole-detection-dataset/data.yaml"

In [ ]:
!python train.py --img {img_shape[0]} --batch 32 --epochs 15 --data {dataset_location} --weights yolov5s.pt --cache

In [ ]:
!python detect.py --weights runs/train/exp2/weights/best.pt --img 640 --conf 0.1 --source {"/kaggle/working/yolov5/pothole-detection-dataset"}/valid/images

In [ ]:
## cli process for developing backend
import subprocess 
cmd="python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.1 --source /kaggle/working/yolov5/pothole-detection-dataset/valid/images --save-txt"
ret_code=subprocess.run(cmd.split(" "),capture_output=True)
print(ret_code.stdout.decode())
print(ret_code.stderr.decode())

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/kaggle/working/yolov5/runs/detect/exp2/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
## save-txt brings report of object detection. this command brings you detected classes and box annotions.
## useful for developing backwend applications.
reports=glob.glob('/kaggle/working/yolov5/runs/detect/exp2/labels/*')
dataframe=pd.concat([pd.read_csv(reports[i],sep=" ",header=None,names=['class','x','y','w','h']) for i in range(len(reports))])
dataframe